In [1]:
from math import sqrt
from operator import itemgetter

import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn import metrics
from MF import PureSingularValueDecomposition

In [49]:
class RegularizedSingularValueDecomposition:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5):

        alpha = 0.01
        d = 20
        
        # Initialize
        U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        mu = self.r
        b_i = self.b_i
        b_u = self.b_u
        
        eta = 0.05
        
        def dJ_sgd(mu, b_u, b_i, U, V, r):
            e = r - (mu + b_u + b_i + U.dot(V))
            return -e, -e + alpha * b_u, -e + alpha * b_i, -e * V + alpha * U, -e * U + alpha * V
        
        for cur_iter in range(n_iter):
            if cur_iter > -1:
                # print(cur_iter)
                f.write(f"{cur_iter}\n")
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_mu, gradient_b_u, gradient_b_i, gradient_U, gradient_V = dJ_sgd(mu, b_u[user], b_i[item], U[user, :], V[item, :], self.R[user, item])
                mu -= eta * gradient_mu
                b_u[user] -= eta * gradient_b_u
                b_i[item] -= eta * gradient_b_i
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                
                # b_u -= eta * gradient_b_u
                # b_i -= eta * gradient_b_i
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)
            if cur_iter > -1:
                print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
                # print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
                f.write(f"{score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test)}\n")
        print("RSVD FINISHED")
        return mu, b_u, b_i, U, V

In [50]:
class Nomu:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5):
        # print("NOMU")
        f.write("NOMU")
        alpha = 0.01
        d = 20
        
        # Initialize
        U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        mu = self.r
        b_i = self.b_i
        b_u = self.b_u
        
        eta = 0.05
        
        def dJ_sgd(mu, b_u, b_i, U, V, r):
            e = r - (mu + b_u + b_i + U.dot(V))
            return -e, -e + alpha * b_u, -e + alpha * b_i, -e * V + alpha * U, -e * U + alpha * V
        
        for cur_iter in range(n_iter):
            if cur_iter > -1:
                # print(cur_iter)
                f.write(f"{cur_iter}\n")
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_mu, gradient_b_u, gradient_b_i, gradient_U, gradient_V = dJ_sgd(mu, b_u[user], b_i[item], U[user, :], V[item, :], self.R[user, item])
                # mu -= eta * gradient_mu
                b_u[user] -= eta * gradient_b_u
                b_i[item] -= eta * gradient_b_i
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                
                # b_u -= eta * gradient_b_u
                # b_i -= eta * gradient_b_i
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)
            if cur_iter > -1:
                print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
                f.write(f"{score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test)}\n")

        return mu, b_u, b_i, U, V

In [66]:
class Nobi:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)

    def gradient_descent(self, n_iter=5):
        f.write("NOBI\n")
        alpha = 0.01
        d = 20

        # Initialize
        U = (np.random.randint(0, 1, size=(self.n, d)) - 0.5) * 0.01
        V = (np.random.randint(0, 1, size=(self.m, d)) - 0.5) * 0.01
        mu = self.r
        b_i = self.b_i
        b_u = self.b_u

        eta = 0.05

        def dJ_sgd(mu, b_u, b_i, U, V, r):
            e = r - (mu + b_u + b_i + U.dot(V))
            return -e, -e + alpha * b_u, -e + alpha * b_i, -e * V + alpha * U, -e * U + alpha * V

        def get_eta(t):
            return 1 / (t + 11)


        for cur_iter in range(n_iter):
            if cur_iter > -1:
                # print(cur_iter)
                f.write(f"{cur_iter}\n")
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]

            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_mu, gradient_b_u, gradient_b_i, gradient_U, gradient_V = dJ_sgd(mu, b_u[user], b_i[item], U[user, :], V[item, :], self.R[user, item])
                # mu -= eta * gradient_mu
                # b_u[user] -= eta * gradient_b_u
                # b_i[item] -= eta * gradient_b_i
                # eta = get_eta(cur_iter * num)
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V

                # b_u -= eta * gradient_b_u
                # b_i -= eta * gradient_b_i
                # U -= eta * gradient_U
                # V -= eta * gradient_V

            eta = eta * 0.95
            ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)
            if cur_iter > -1:
                print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
                f.write(f"{score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test)}\n")

        return mu, b_u, b_i, U, V

In [ ]:
class MatrixFactorization:
    def __init__(self, records_train, records_test):
        records = np.vstack([records_train, records_test])
        self.n = len(np.unique(np.sort(records[:, 0])))
        self.m = len(np.unique(np.sort(records[:, 1])))

        # Initial R
        self.R = np.zeros([self.n, self.m], dtype=np.int32)

        for record in records_train:
            self.R[record[0], record[1]] = record[2]

        # Initial indicator
        y = np.where(self.R, 1, 0)
        y_user = np.sum(y, axis=1)
        y_item = np.sum(y, axis=0)

        # Global average of rating
        self.r = np.sum(self.R) / np.sum(y)

        # average rating of user
        self.r_u = np.where(y_user,
                            np.sum(self.R, axis=1) / y_user,
                            self.r)

        # average rating of item
        self.r_i = np.where(y_item,
                            np.sum(self.R, axis=0) / y_item,
                            self.r)

        # bias of user
        self.b_u = np.where(y_user,
                            np.sum(y * (self.R - self.r_i), axis=1) / y_user,
                            0)

        # bias of item
        self.b_i = np.where(y_item,
                            np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
                            0)
        
    def gradient_descent(self, n_iter=5000):


        R_svd = np.where(self.R == 0,
                          np.zeros(shape=self.R.shape) + self.r_u.reshape(-1, 1),
                          self.R)
        # SVD
        U, s, VT = svd(R_svd)

        d = 20
        Sigma = np.zeros([d, d])
        for i in range(d):
            Sigma[i][i] = s[i]
        
        U = U[:, :d].dot(Sigma)
        V = VT[:d, :].T
        
        def performance():
            return U.dot(V.T)[records_test[:, 0], records_test[:, 1]]
        ratings_predict_rsvd = performance()
        print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))
        print("GG")
        eta = 0.0001
        alpha = 0.0001
        mu = self.r
        
        def dJ_sgd(U, V, r):
            e = r - U.dot(V)
            return -e * V + alpha * U, -e * U + alpha * V

        for cur_iter in range(n_iter):
            print(cur_iter)
            ratings = np.where(self.R != 0)
            num = len(ratings[0])
            indexes = np.random.permutation(num)
            users = ratings[0][indexes]
            items = ratings[1][indexes]
            
            for i in range(num):
                user = users[i]
                item = items[i]
                gradient_U, gradient_V = dJ_sgd(U[user, :], V[item, :], self.R[user, item])
                
                U[user, :] -= eta * gradient_U
                V[item, :] -= eta * gradient_V
                ratings_predict_rsvd = performance()
                if not ratings_predict_rsvd.min() > -10:
                    print(i)
                    break
                # U -= eta * gradient_U
                # V -= eta * gradient_V
                
            eta = eta * 0.9
            ratings_predict_rsvd = performance()
            # print(ratings_predict_rsvd.min() > -10)
            # print(ratings_predict_rsvd[:20])
            print(score(np.clip(ratings_predict_rsvd, 1, 5), ratings_test))

        return

In [5]:
def score(ratings_test, ratings_predict):
    return [round(sqrt(metrics.mean_squared_error(ratings_test, ratings_predict)), 4),
            round(metrics.mean_absolute_error(ratings_test, ratings_predict), 4)]
def performance(mu, b_u, b_i, U, V, records_test):
        return mu + b_u[records_test[:, 0]] + b_i[records_test[:, 1]] + U.dot(V.T)[records_test[:, 0], records_test[:, 1]]

In [6]:
# Load the records
records_train = np.loadtxt('../data/ml-100k/u1.base', dtype=np.int32)
records_test = np.loadtxt('../data/ml-100k/u1.test', dtype=np.int32)

# Preprocess
records_train[:, :2] -= 1
records_test[:, :2] -= 1
ratings_test = records_test[:, 2]
records = np.vstack([records_train, records_test])

In [7]:
%%time
psvd = PureSingularValueDecomposition(records_train, records_test)

F:\recommender-algorithm-implementation\Matrix Factorization\MF.py:39: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,


Wall time: 569 ms


In [8]:
ratings_predict = psvd.performance(records_test)
# ratings_predict.max()

print(score(np.clip(ratings_predict, 1, 5), ratings_test))
# score(np.clip(ratings_predict, 1, 5), ratings_test)

[1.017, 0.8058]


In [43]:
%%time
rsvd = RegularizedSingularValueDecomposition(records_train, records_test)

mu, b_u, b_i, U, V = rsvd.gradient_descent(100)
ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)


nomu = Nomu(records_train, records_test)
mu, b_u, b_i, U, V = nomu.gradient_descent(100)


nobi = Nobi(records_train, records_test)

nobi.gradient_descent(100)

<ipython-input-31-e93682ee952f>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-31-e93682ee952f>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
<ipython-input-37-16130be8d572>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-37-16130be8d572>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
<ipython-input-33-8e2daa1e15ed>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-33-8e2daa1e15ed>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,


RSVD FINISHED
Wall time: 5min 35s


(3.52835,
 array([ 1.08952097e-01,  1.47292722e-01, -2.96089856e-01,  9.95861591e-01,
        -3.07979273e-01, -2.13216449e-01,  4.17613261e-01,  1.55244582e-01,
         4.37816226e-01,  3.93208874e-01, -3.14850500e-02,  4.85574031e-01,
        -2.04277013e-01,  4.53709133e-01, -3.78670265e-01,  6.83629005e-01,
        -4.51756604e-01,  1.90466363e-01,  5.30517075e-02, -3.50328603e-01,
        -2.61375162e-01, -3.90854408e-02, -3.90906427e-02,  6.23449095e-01,
         1.82629893e-01, -4.87171487e-01, -1.25888340e-01,  1.63794270e-01,
         1.57708335e-01,  3.20418408e-01,  2.39244734e-01, -8.13165066e-02,
         4.79262630e-01,  6.12226446e-01, -1.30150467e-01,  1.10085655e+00,
         5.97272814e-02,  7.70034649e-01,  2.41236551e-01, -7.36980243e-01,
        -1.66686506e-02,  9.72784422e-02,  1.66324018e-01, -3.15177347e-02,
         2.12575771e-01,  5.27085214e-01,  1.55333386e-01, -1.60959765e-01,
        -6.48810133e-01,  2.84395930e-02, -2.56067081e-01,  6.52560734e-01,
  

In [42]:
f = open("results.txt", 'w')

In [25]:
a = 5
f.write(f"{a}")
f.write('\n')
f.close()

In [35]:
%%time
mf = MatrixFactorization(records_train, records_test)

mf.gradient_descent(70)

<ipython-input-4-6f5355759d99>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-4-6f5355759d99>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,


[1.017, 0.8058]
GG
0
879
[1.5149, 1.1714]
1
0
[1.5149, 1.1715]
2
0
[1.5149, 1.1715]
3
0
[1.5149, 1.1716]
4
0
[1.5149, 1.1716]
5
0
[1.5156, 1.1722]
6
0
[1.5159, 1.1728]
7
0
[1.5163, 1.1733]
8
0
[1.5163, 1.1733]
9
0
[1.5161, 1.1731]
10
0
[1.516, 1.1729]
11
0
[1.516, 1.1729]
12
0
[1.5153, 1.1722]
13
0
[1.5154, 1.1724]
14
0
[1.5154, 1.1724]
15
0
[1.5157, 1.1727]
16
0
[1.5157, 1.1727]
17
0
[1.5158, 1.1728]
18


KeyboardInterrupt: 

In [44]:
f.close()

In [53]:
f = open("results_nobi_2.txt", 'w')
nobi = Nobi(records_train, records_test)

nobi.gradient_descent(100)
f.close()

<ipython-input-52-8edc6e2cfb79>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-52-8edc6e2cfb79>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,


[0.9617, 0.7601]
[0.95, 0.7487]
[0.9435, 0.7406]
[0.943, 0.7392]
[0.9426, 0.7391]
[0.9432, 0.7394]
[0.9432, 0.7388]
[0.9429, 0.7385]
[0.9435, 0.7389]
[0.9438, 0.7388]
[0.944, 0.739]
[0.9445, 0.7392]
[0.9445, 0.7391]
[0.9443, 0.7389]
[0.9446, 0.7392]
[0.945, 0.7392]
[0.945, 0.739]
[0.9453, 0.7393]
[0.9454, 0.7393]
[0.9454, 0.7393]
[0.9455, 0.7393]
[0.9455, 0.7393]
[0.9457, 0.7393]
[0.9457, 0.7393]
[0.9458, 0.7394]
[0.9458, 0.7394]
[0.9459, 0.7394]
[0.9459, 0.7394]
[0.9459, 0.7394]
[0.9459, 0.7395]
[0.946, 0.7395]
[0.946, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463

KeyboardInterrupt: 

In [51]:
%%time
f = open("results_others.txt", 'w')

rsvd = RegularizedSingularValueDecomposition(records_train, records_test)

mu, b_u, b_i, U, V = rsvd.gradient_descent(100)
ratings_predict_rsvd = performance(mu, b_u, b_i, U, V, records_test)


nomu = Nomu(records_train, records_test)
mu, b_u, b_i, U, V = nomu.gradient_descent(100)
f.close()

<ipython-input-49-a9fdcd92662e>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-49-a9fdcd92662e>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,
<ipython-input-50-1b751bfdfbcb>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-50-1b751bfdfbcb>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,


[0.967, 0.7609]
[1.0189, 0.7831]
[0.9702, 0.7561]
[0.9638, 0.7557]
[0.9601, 0.7617]
[0.9469, 0.7415]
[0.9404, 0.7367]
[0.9386, 0.734]
[0.9389, 0.7326]
[0.9361, 0.7297]
[0.9339, 0.7327]
[0.9357, 0.7279]
[0.9359, 0.7391]
[0.9326, 0.7281]
[0.9319, 0.7312]
[0.9312, 0.7308]
[0.9314, 0.7291]
[0.9373, 0.7273]
[0.9327, 0.7274]
[0.9317, 0.7276]
[0.9389, 0.7276]
[0.9319, 0.727]
[0.9315, 0.7272]
[0.931, 0.7305]
[0.9359, 0.7268]
[0.9311, 0.7314]
[0.931, 0.727]
[0.9323, 0.7266]
[0.9318, 0.7267]
[0.9305, 0.7288]
[0.9305, 0.7285]
[0.9306, 0.7278]
[0.9306, 0.7298]
[0.9307, 0.7274]
[0.9306, 0.7297]
[0.9306, 0.7298]
[0.9305, 0.7287]
[0.9306, 0.7294]
[0.9309, 0.7273]
[0.9308, 0.7275]
[0.9307, 0.7277]
[0.9307, 0.7275]
[0.9308, 0.7274]
[0.9306, 0.7277]
[0.9306, 0.728]
[0.9305, 0.7285]
[0.9307, 0.7276]
[0.9305, 0.7283]
[0.9305, 0.7287]
[0.9305, 0.7281]
[0.9306, 0.7278]
[0.9306, 0.7278]
[0.9305, 0.7284]
[0.9307, 0.7276]
[0.9308, 0.7273]
[0.9307, 0.7275]
[0.9307, 0.7276]
[0.9306, 0.7278]
[0.9305, 0.7281]
[0.9

In [67]:
%%time
f = open("results_nobi_2.txt", 'w')
nobi = Nobi(records_train, records_test)

nobi.gradient_descent(100)
f.close()

<ipython-input-66-4c5406b3ac98>:28: RuntimeWarning: invalid value encountered in true_divide
  np.sum(self.R, axis=0) / y_item,
<ipython-input-66-4c5406b3ac98>:38: RuntimeWarning: invalid value encountered in true_divide
  np.sum(y * (self.R - self.r_u.reshape(-1, 1)), axis=0) / y_item,


[0.962, 0.7604]
[0.9615, 0.7599]
[0.9569, 0.7554]
[0.9463, 0.7443]
[0.9428, 0.7404]
[0.942, 0.7392]
[0.9421, 0.7388]
[0.9418, 0.7379]
[0.9428, 0.7385]
[0.9427, 0.7384]
[0.9422, 0.7381]
[0.9429, 0.7383]
[0.9433, 0.7384]
[0.9435, 0.7385]
[0.9435, 0.7387]
[0.9436, 0.7385]
[0.9439, 0.7386]
[0.9441, 0.7388]
[0.9441, 0.7388]
[0.9443, 0.7388]
[0.9445, 0.7389]
[0.9447, 0.7389]
[0.9446, 0.7389]
[0.9447, 0.7389]
[0.9449, 0.739]
[0.9452, 0.7391]
[0.9453, 0.7392]
[0.9454, 0.7392]
[0.9453, 0.7392]
[0.9453, 0.7391]
[0.9455, 0.7392]
[0.9455, 0.7393]
[0.9457, 0.7393]
[0.9457, 0.7393]
[0.9457, 0.7393]
[0.9457, 0.7393]
[0.9458, 0.7393]
[0.9459, 0.7394]
[0.9459, 0.7394]
[0.946, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7395]
[0.9461, 0.7395]
[0.9462, 0.7395]
[0.9462, 0.7396]
[0.9462, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9463, 0.7396]
[0.9464, 0.7396]
[0.9464, 0.7396]
[0.9464, 0.7396]
[0.9464, 0.7396]
[0.9465, 0.7396]
[0.9465, 0.7397]
[0.9465, 0.7397]
[0.9465, 0.7397]
[0.9465, 0.7397]
[0.9465, 0.7397]
[